## Zadania do wykonania

### Zadanie 1. Zaprojektuj klasę dla $kd$-drzewa.

### Zadanie 2. Zaimplementuj wzorzec projektowy łańcuch odpowiedzialności na przykładzie obsługi żądania _http_ (symulacja), w którym przed faktycznym kodem obsługi błędu ma zostać sprawdzone czy użytkownik może wysłać danego typu żądanie, czy żądanie nie dotyczy pliku, czy liczba żądań na minutę nie jest przekroczona, czy liczba żądań na minutę nie jest przekroczona dla zalogowanego użytkownika, czy przesłany formularz nie jest próbą `sql incjection`.

### Zadanie 3. Za pomocą dowolnego wzorca projektowego zaimplementuj mechanizm sprawdzający poprawność wyrażenia postaci:
* a + b = c (poprawne),
* (a + b = c (niepoprawne),
* a + = c (niepoprawne).

In [48]:
# zadanie 1

from collections import namedtuple 
from operator import itemgetter 
from pprint import pformat

class Tree(namedtuple("Tree", "loc left right")): 
    def __repr__(self): 
        return pformat(tuple(self))
    
def kdtree(kdList, deep = 0):
    if not kdList:
        return None
    pivot = deep % len(kdList[0])
    kdList.sort(key=itemgetter(pivot))
    med = len(kdList) // 2 
    return Tree(
        loc=kdList[med],
        left=kdtree(kdList[:med], deep + 1),
        right=kdtree(kdList[med + 1 :], deep + 1)
    )


print(kdtree([(8, 0), (1, 4), (7, 7), (1, 3), (5, 4)]))

((5, 4),
 ((1, 4), ((1, 3), None, None), None),
 ((7, 7), ((8, 0), None, None), None))


In [67]:
# zadanie 2


from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any, Optional


class Handler(ABC):

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        pass


class AbstractHandler(Handler):

    _next_handler: Handler = None

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)

        return None

class czyMozeWyslacZadanie(AbstractHandler):
    def handle(self, request: Any) -> str:
        if "http" in request:
            return True
        else:
            return super().handle(request)
    
class czyZadanieNieDotyczyPliku(AbstractHandler):
    def handle(self, request: Any) -> str:
        if not "c:\\" in request:
            return True
        else:
            return super().handle(request)
        
class czyLiczbaZadanNaMinNieJestPrzekroczona(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "x":
            return True
        else:
            return super().handle(request)
   
class czyLiczbaZadanNaMinNieJestPrzekroczonaDlaZalogowanego(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "x":
            return True
        else:
            return super().handle(request)
   
class czyNieSqlIncjection(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "x":
            return True
        else:
            return super().handle(request)


def client_code(handler: Handler) -> None:
    
    result = handler.handle('http://www.onet.pl/')
    print(f"{result}")

    
cond1 = czyMozeWyslacZadanie()
cond2 = czyZadanieNieDotyczyPliku()
cond3 = czyLiczbaZadanNaMinNieJestPrzekroczona()
cond4 = czyLiczbaZadanNaMinNieJestPrzekroczonaDlaZalogowanego()
cond5 = czyNieSqlIncjection()

client_code(cond1)
client_code(cond2)
client_code(cond3)
client_code(cond4)
client_code(cond5)

True
True
None
None
None


In [45]:
# zadanie 3


# facade design pattern

# VALID SIGNS:
    # alphabet letters
    # + - * / =
    # [] {} ()
    
# NOT ALLOWED SIGNS:
    # numbers
        
        
class Variables:
    
    def __init__(self,form):
        self.x = form
    
    def checkSize(self):
        lenx = 0
        for str in self.x:
            if str.isalpha():
                lenx += 1
                
        return lenx;
    
    
class Signs:
    
    def __init__(self,form):
        self.x = form
    
    def checkSize(self):
        lenx = 0
        for str in self.x:
            if (str=='+' or 
                str=='-' or 
                str=='/' or 
                str=='*' or 
                str=='='):
                lenx += 1
                
        return lenx;

    
class OtherConditions:
    
    def __init__(self,form):
        self.x = form
        self.noneBrackets = ''
        
    def newForm(self):
        for str in self.x:
            if not(str=='(' or 
                   str==')' or 
                   str=='[' or 
                   str==']' or 
                   str=='{' or 
                   str=='}'):
                self.noneBrackets += str 
        return self.noneBrackets;
    
    def checkPlaces(self):
        y = self.newForm()
        for ele in range(0, len(y)):
            if ((y[ele].isalpha()) and (ele % 2 == 0)):
                continue
            elif ((y[ele]=='+' or 
                   y[ele]=='-' or 
                   y[ele]=='/' or 
                   y[ele]=='*' or 
                   y[ele]=='=') 
                  and (ele % 2 != 0)):
                continue
            else:
                return False
        return True
    
    
class Equal:  
    
    def __init__(self,form):
        self.x = form
    
    def amountEqual(self):
        c = False
        for str in self.x:
            if (c & (str=='=')):
                return False
            elif str=='=':
                c = True
        if not c:
            return False
        return True
    
    def condEqBrackets(self):
        open = False
        eq = False
        for str in self.x:
            if (open & eq & 
                (str == ')' or 
                 str == ']' or 
                 str == '}')):
                return False
            elif (open & (str=='=')):
                eq = True    
            elif (str == '(' or str == '[' or str == '{'):
                open = True
        return True

    
class Brackets:
    
    def __init__(self,form):
        self.x = form
    
    def checkBrackets(self):
        for ele in range(0, len(self.x)):
            if (self.x[ele]=='(' or self.x[ele]=='{' or self.x[ele]=='['):
                if (ele == 0):
                    if (self.x[ele+1].isalpha()):
                        continue
                    else:
                        return False
                elif ((self.x[ele-1]=='+' or 
                       self.x[ele-1]=='-' or 
                       self.x[ele-1]=='*' or 
                       self.x[ele-1]=='/' or 
                       self.x[ele-1]=='=') 
                      and self.x[ele+1].isalpha()):
                    continue
                else:
                    return False
            elif (self.x[ele]==')' or self.x[ele]=='}' or self.x[ele]==']'):
                if (ele == len(self.x)-1):
                    if (self.x[ele-1].isalpha()):
                        continue
                    else:
                        return False
                elif ((self.x[ele+1]=='+' or 
                       self.x[ele+1]=='-' or 
                       self.x[ele+1]=='*' or 
                       self.x[ele+1]=='/' or 
                       self.x[ele+1]=='=') 
                      and self.x[ele-1].isalpha()):
                    continue
                else:
                    return False
        return True
                
    def checkAmountBrackets(self):
        listOpen = 0
        listClose = 0
        for str in self.x:
            if (str=='(' or 
                str=='[' or 
                str=='{'):
                listOpen += 1
            elif (str==')' or 
                  str==']' or 
                  str=='}'):
                listClose += 1
                
        if listOpen == listClose:
            return True
        
        return False
    
    def bracketsCond(self):
        open = False
        for str in self.x:
            if open & (str=='(' or str=='[' or str=='{'):
                return False
            elif (str=='(' or str=='[' or str=='{'):
                open = True
        return True

    def closeOpen(self):
        brackets = []
        for str in self.x:
            if (str=='(' or str=='[' or str=='{'):
                brackets.append('open')
            if (str==')' or str==']' or str=='}'):
                brackets.append('close')
        for element in range(0, len(brackets)):
            str = brackets[element]
            if (str=='open') and element%2!=0:
                return False
            if (str=='close') and element%2==0:
                return False
        return True
        
        
class Equation:

    def __init__(self,form):
        self.eq = form
        noSpaces = form.replace(' ','')
        self.var = Variables(noSpaces)
        self.sign = Signs(noSpaces)
        self.cond = OtherConditions(noSpaces)
        self.bracket = Brackets(noSpaces)
        self.equal = Equal(noSpaces)
    
    def startTest(self):
        cond1 = self.var.checkSize()-1 == self.sign.checkSize() # checking amount of signs
        cond2 = self.cond.checkPlaces() # checking places of signs
        cond3 = self.bracket.checkBrackets() # custom condition
        cond4 = self.bracket.checkAmountBrackets() # is ( amount equal to ) amount
        cond5 = self.equal.amountEqual() # only one =
        cond6 = self.equal.condEqBrackets() # not allowed -> ( = )
        cond7 = self.bracket.bracketsCond() # double open brackets
        cond8 = self.bracket.closeOpen() # close bracket without open
        
        if cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8:
            print(f'Correct equation: {self.eq}')
        else:
            print(f'Uncorrect equation: {self.eq}')
        
        
equ1 = Equation('a / r = (c + x)*u + c')
equ1.startTest()

equ2 = Equation('a / r (= c + x)*u + c')
equ2.startTest()

equ3 = Equation('a / r = c + x*u/u/ + c')
equ3.startTest()

Correct equation: a / r = (c + x)*u + c
Uncorrect equation: a / r (= c + x)*u + c
Uncorrect equation: a / r = c + x*u/u/ + c
